In [1]:
import os
import csv
import sys
import re
import string
import nltk
import codecs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import bigrams 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet as wn

#from nltk.stem.wordnet import WordNetLemmatizer
#from nltk import PorterStemmer
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.externals import joblib

# import mpld3

In [2]:
def removeTAGS(x):
    x = re.sub(r'\s+',r' ',x)
    x = re.sub('<code>.*?code>', r' ', x)
    x = re.sub('<.*?>', r' ', x)
    return(x)

In [3]:
def removeUrls(x):
    x = x.lower()
    x = re.sub('http.*?\s', r' ', x)
    return(x)

In [4]:
def removePunctuation(x):
    x = x.lower()
    x = re.sub(r'[^\x00-\x7f]',r' ',x) #For Characters like Â
    x = re.sub(r'\s+',r' ',x) #Multiple Spaces, tabs, carriage returns to single space
    return re.sub("["+string.punctuation+"]", " ", x)

In [5]:
def removeNos(x):
    x = re.sub(r'[^a-z\s]', r' ', x)
    x = re.sub(r'\s+',r' ',x)
    x = re.sub(r'(\s\w{1,2})+\s', r' ',x)
    x = re.sub(r'\s+',r' ',x)
    return(x)

In [31]:
stops = set(stopwords.words("english"))
stops.update(["would", "get", "like", "likely", "using", "know", "question", "use", "get", "possible" , "much", "find", "anyone"])
stops.update(["tell", "know", "another", "various", "also", "etc", "around", "vs", "used", "could", "without", "way", "new"])
stops.update(["need", "known", "make", "makes", "made", "file", "downloaded", "download", "one", "two", "even", "far", "thus"])
stops.update(["every", "said", "call", "tell", "tells", "called", "per", "though", "less","since", "problem", "answer", "answers"])
stops.update(["small", "vast","therefore", "must", "simply", "put", "less", "true", "talk", "said", "per", "related"])
stops.update(["need", "needed", "variety", "cause", "causes", "later", "unrelated", "entire", "saying", "give", "gave", "false"])         
stops.update(["show", "shows", "however", "previous", "previously", "average", "mean", "think", "thought", "little", "according"])
stops.update(["number", "view", "views", "viewed", "send", "sent", "may", "might", "right", "wrong", "high", "low" ])
stops.update(["min", "see", "saw", "sees", "someone", "please", "explain", "explains", "text", "texts", "indicate", "indicates"])              
stops.update(["begin", "began", "take", "taken", "array", "arrive", "conclusion", "conclusions", "hline", "amp", "edit", "end"])
stops.update(["thank", "thanks", "assistance", "end", "error", "left", "right", "suppose", "sample", "given", "best", "guess"])
stops.update(["figure", "matlab", "parameters", "parameter","paper", "papers","whether", "understand",  "quite", "work", "work"]) 
stops.update(["clears", "obtain", "obtains", "obtained", "form", "forms", "find", "found", "example", "examples"])
stops.update(["better", "nothing", "different", "well", "say", "says", "seem", "huge", "large", "definition", "meaning"])
stops.update(["larger", "many", "seems", "pretty", "large", "never", "big", "bigger", "yet", "detail", "details", "nbsp"])
stops.update(["cdot", "vec", "table", "thing", "either", "wait", "overview", "something", "else", "came", "across"])
stops.update(["wikipedia", "article", "feel", "free", "stack", "exchange", "quick", "google", "anything", "read", "wiki"])
stops.update(["appreciate", "help", "got", "thin", "thinking", "scholar", "several", "times", "currently", "working", "studied"])
stops.update(["really", "want", "text", "sure", "correct", "simple", "language", "wanted", "ask", "read", "lot", "takes"])
stops.update(["let", "consider", "year", "years", "ago", "long", "term"])
#
stops.update(["look", "looks", "looked", "case", "cases", "wonder", "wondering", "come", "comes", "change", "changes", "changing" ])
stops.update(["look","changed", "differ", "effect", "follow", "function", "happen", "inform", "look", "peopl", "produc" ])
stops.update(["process","read", "studi", "reason", "inform", "result", "similar", "time", "type"])
stops.update(["abl", "actual", "alway", "amount", "appear", "back", "becom", "book", "clear", "common", "compar", "contain"])
stops.update(["determin", "develop", "due", "contain", "exact", "explan", "express", "first", "general", "good", "idea", "identifi"])
stops.update(["includ", "increas", "interest", "kind", "learn", "lead", "link", "level", "measur", "method", "name", "often"])
stops.update(["place", "point", "rather", "provid", "refer", "respons", "search", "seen", "select", "requir", "relat", "recent"])
stops.update(["start", "still", "within", "certain", "complet", "research", "day", "enough", "least", "direct", "test"])
stops.update(["base", "exist", "factor", "grow", "live", "mormal", "occur", "order","suggest", "present", "part", "particular"])
stops.update(["via","word", "whole", "valu", "usual", "turn", "total", "step", "sourc", "sort", "size", "side", "short", "set"])
stops.update(["second", "run", "remov", "regard", "possibl", "perhap", "note", "move", "mayb", "main", "lower", "list"])
stops.update(["keep", "last", "involv", "insid", "import", "higher", "go", "group", "generat", "fact", "expect", "done"])
stops.update(["obvious","origin","popul", "pathway", "creat", "assum", "allow", "advantag", "caus", "color", "creat", "defin"])
stops.update(["describ", "consid", "avail", "condit", "confus", "depend", "eat", "heard", "instead", "length", "locat", "multipl"])
stops.update(["limit", "imagin", "cours", "area", "depend"])
stops.update(["curious","close", "believ", "alreadi", "affect", "tri"])
stops.update(["advanc", "bind", "bit", "calcul", "code", "mention", "observ", "tri", "theori", "rate", "non", "notic", ])
stops.update(["individu", "infec", "normal", "pair", "perform", "physic", "pictur", "post", "probabl", "posit", "specif", "theori"])
stops.update(["comput", "state", "analysi", "addit", "activ", "concentr", "control", "current", "cycl", "evid", "imag"])
stops.update(["infect", "interact", "line", "singl", "signific", "surviv"])
stops.update(["abil", "act", "action", "add", "age", "almost", "along", "altern", "although", "among", "appreci", "background"])
stops.update(["appli", "associ", "basic", "behind", "carri", "complex", "concept", "continu", "cross", "cut", "damag"])
stops.update(["decreas", "detect", "discuss", "effic", "especi", "estim", "ever", "experience", "experiences", "extract"])
stops.update(["format", "great", "hard", "hope", "hour", "initi", "issu", "lack", "leav", "longer", "materi", "near", "next"])
stops.update(["old", "open", "outsid", "page", "pass", "prefer", "pressur", "product", "reach", "real", "red", "releas"])
stops.update(["remain", "rest", "sampl", "separ", "site", "sometim", "stop", "strand", "strong", "subject", "three", "today"])
stops.update(["togeth", "topic", "trait", "transcript", "typic", "white", "target", "subject","solut","sound"])
stops.update(["claim", "connect", "effici", "frequenc", "instanc", "knowledg", "lab", "negat", "pattern", "necessari", "matter"])
stops.update(["life", "negat", "period", "purpos", "random", "rang","reduc", "regul", "replic", "variat", "light", "kill"])
stops.update(["prevent", "standard", "role", "signal"])
stops.update(["account", "air", "ad", "align", "angl", 
"angular", "appar", "applic", "approach",
"approxim", "arbitrari", "argument", "assumpt",
"attract", "away", "axi", "ball", "bar", "basi",
"beam", "beta", "bodi", "boundari", 
"break", "center", "classically", "classicality",
"coeffici", "collis", "combin", "compon", 
"consist", "constant", "construct", "context",
"coordin", "correspond", "cos", "curv", "definit",
"degre", "delta", "densiti", "deriv", "diagram",
"dimens", "dimension", "distribut", "dot", "drop", "easi", "emit", "ensilon", "equal",
"equat", "equival", "event", "everyth", "expand",
"expans", "extern", "fall", "faster", "fielding",
"fielded", "final", "finit", "fix", "flow", "frac", 
"formula", "frame", "full", "fundament",
"gas", "gaug", "goe", "greater", "ground",
"half", "hand", "hat", "height", "hence", "hit",
"hold", "holed", "hoizont", "ideal", "ident",
"ignor", "impli", "independ", "infinit", "infti", 
"int", "integr", "intern", "interpret", "introduc",
"intuit", "invari", "lambda", "langl", "law", 
"linear", "local", "magnitud", "mass", "mathbf",
"mathcal", "mathrm", "matrix", "mathemat",
"maximum", "metric", "miss", "nabla", "natur",
"net", "object", "omega", "oper", "opposit",
"parallel", "partial", "path", "perfect", "perpendicular", "phase", "phi", "plane",
"potenti", "power", "precis", "predict", 
"principl", "project", "proof", "propag",
"proper", "properti", "proport", "prove", "psi",
"qft", "quantiti", "radius", "rangl", "ray",
"reflect", "regional", "relationship",
"repres", "represent", "respect", "rho", 
"rightarrow", "rule", "satisfi", "scalar",
"scale", "scatter", "section", "sens", 
"shape", "shown", "sigma", "simul",
"sin", "situat", "smaller", "solid", "solv",
"somehow", "spacing", "spaced", "spacings",
"special", "sphere", "spheric", "spin", "sqrt",
"squar", "star", "statement", "stationari", 
"straight", "structur", "sum", "sun", "suppos",
"symmetri", "textbook", "theoret", "theta", "top", 
"toward", "transfer", "transform", "travel", 
"troubl", "uniform", "unit", "universal",
"university", "universality", "universities",
"valid", "vari", "variabl", "vector", "vertic",
"volum", "water", "weight", "world", "write", 
"written", "yes", "zero"
])
stop_words = pd.DataFrame(list(stops), columns = ["words"])
stop_words = stop_words.sort_values(['words'])
stop_words.to_csv("stop_words.csv", index= False)
#stop_words.sort_values(['frequency'], ascending=[False])

In [32]:
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

In [400]:
#similarity2("feline", "cat")

[0.5, 0.08333333333333333, 0.08333333333333333, 0.058823529411764705, 0.0625, 0.058823529411764705, 0.5, 0.043478260869565216]


1

In [8]:
def similarity(w1, w2):
    synsets1 = wn.synsets(w1)
    synsets2 = wn.synsets(w2)
    sim_scores = []
    for synset1 in range(len(synsets1)):
        for synset2 in range(len(synsets2)):
            if (synsets1[synset1].wup_similarity(synsets2[synset2], simulate_root = False) != None) \
            and synsets1[synset1].wup_similarity(synsets2[synset2], simulate_root = False) >= 0 :
                sim_scores.append(synsets1[synset1].wup_similarity(synsets2[synset2], simulate_root = False))
    print(sim_scores)
    if len(sim_scores) == 0:
        return 0
    else:
        return 1

In [9]:
def check_synonym(word, word2):
    l_syns = list()
    synsets = wn.synsets(word)
    for synset in synsets:
        if word2 in synset.lemma_names():
            l_syns.append( (word, word2) )
    return l_syns

In [10]:
def similarity2(w1, w2):
    synsets1 = wn.synsets(w1)
    synsets2 = wn.synsets(w2)
    sim_scores = []
    for synset1 in range(len(synsets1)):
        for synset2 in range(len(synsets2)):
            if (synsets1[synset1].path_similarity(synsets2[synset2]) != None) \
            and synsets1[synset1].path_similarity(synsets2[synset2]) >= 0.5 :
                sim_scores.append(synsets1[synset1].path_similarity(synsets2[synset2]))
    #print(sim_scores)
    if len(sim_scores) == 0:
        return 0
    else:
        return 1

In [11]:
#check_synonym("molecule","feline")

In [12]:
#syns = wn.synsets("program")
#syns

In [13]:
#cat = wn.synset('feline.n.01')
#cat.hyponyms()

In [14]:
#synsets1 = wn.synsets("feline")
#synsets1
#synsets1[2].lemma_names()

In [15]:
#synsets2 = wn.synsets("evolutionary")
#synsets2

In [16]:
#print(synsets1[1].wup_similarity(synsets2[0]))

In [17]:
# #similarity("genetic", "genome")
# synsets1 = wn.synsets("cat")
# print(synsets1)
# synsets2 = wn.synsets("feline")
# print(synsets2)
# type(synsets1)

# if synsets1[0] == synsets2[0]:
#     print("True")
# else:
#     print("False")

In [297]:
# type(synsets1[0])

nltk.corpus.reader.wordnet.Synset

In [298]:
# type(w1)

nltk.corpus.reader.wordnet.Synset

In [277]:
# for ss in wn.synsets('neuron'):
#     print(ss.name(), ss.lemma_names())

nerve_cell.n.01 ['nerve_cell', 'neuron']


In [288]:
# from itertools import chain
# synonyms = wn.synsets('neuron')
# set(chain.from_iterable([word.lemma_names() for word in synonyms]))

{'nerve_cell', 'neuron'}

In [18]:
# w1 = wn.synset('gene.n.01')
# w2 = wn.synset('genetic.n.01')
# print(w1.wup_similarity(w2))

In [33]:
biology = pd.read_csv("test.csv")

In [34]:
biology['combine'] = " " + biology['title'] + " " + biology['content'] + " "
biology['combiny'] = biology['combine'].map(removeTAGS)
biology['combiny'] = biology['combiny'].map(removeUrls)
biology['combiny'] = biology['combiny'].map(removePunctuation)
biology['combiny'] = biology['combiny'].map(removeNos)
biology['combiny'] = biology['combiny'].map(removeStopwords)

In [ ]:
# biology["tag_list"] = biology["tags"].map(lambda x: x.split())
# tags = biology["tag_list"].tolist()
# all_tags = [j for i in range(len(tags)) for j in tags[i]]
# count = pd.DataFrame(all_tags, columns=["word"])
# count1 = count.groupby(["word"]).size().reset_index(name='count')
# count1.sort_values(by=["count"], inplace = True, ascending = False)

In [35]:
stemmer = SnowballStemmer("english")
combiny = biology['combiny'].tolist()

In [36]:
def tokenize_and_stem(text):
    tokens = [token for sent in nltk.sent_tokenize(text) for token in nltk.word_tokenize(sent)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search(r'[a-zA-Z]',token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]      
    return stems

def tokenize_only(text):
    tokens = [ token for sent in nltk.sent_tokenize(text) for token in nltk.word_tokenize(sent)]
    filtered_token = []
    
    for token in tokens:
        if re.search(r'[a-zA-Z]', token):
            filtered_token.append(token)
    return filtered_token

In [37]:
filtered_text=[]
stemmed_text = []
for i in combiny:
    stemmed_text.extend(tokenize_and_stem(i))
    filtered_text.extend(tokenize_only(i))


In [ ]:
#len(stemmed_text)

In [38]:
len(filtered_text)

3330665

In [39]:
len(set(filtered_text))

56816

In [40]:
stemmed_dict = pd.DataFrame({'stem':stemmed_text, 'word':filtered_text})
stemmed_dict_freq = stemmed_dict.groupby(["stem","word"]).size().reset_index(name='count')
stemmed_dict_freq["length"] = stemmed_dict_freq["word"].apply(len)
stemmed_dict_freq.sort_values(by = ["count","length"], inplace= True, ascending = [False, False] )
stemmed_dict_freq.iloc[0:4,:]

,stem,word,count,length
16068,energi,energy,38139,6
18313,field,field,27987,5
16468,equat,equation,22295,8
19066,forc,force,22156,5


In [41]:
k = stemmed_dict_freq.set_index(["stem"])
def fa(x):
    if type(x) == type("word") :
        return([x])
    else:
        return x.tolist()
k_1 = {w : fa(k.ix[w,0]) for w in set(stemmed_text)}
#k_1["tri"]

In [42]:
lookup = stemmed_dict_freq.groupby(["stem"]).first().reset_index()
lookup2 = lookup.set_index(["stem"])
#lookup2.drop()

In [43]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.015, stop_words=stops,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(0,1))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(combiny) #fit the vectorizer to content

print(tfidf_matrix.shape)

Wall time: 1min 59s
(81926, 93)


In [44]:
terms = tfidf_vectorizer.get_feature_names()
type(terms)
terms
#terms.apply(lambda x: lookup2.ix[terms[x]]['word'])
words = [lookup2.ix[terms[x]]['word'] for x in range(len(terms))]
words

for a in terms:
    print(a, k_1[a])


acceler ['acceleration', 'accelerating', 'accelerate', 'accelerated', 'accelerates', 'accelerations', 'accelerator', 'accelerators', 'accelerational', 'acceleratingly']
alpha ['alpha', 'alphas']
amplitud ['amplitude', 'amplitudes', 'amplitud', 'amplituded']
atom ['atom', 'atoms', 'atomic', 'atomically', 'atomizers', 'atomizer', 'atomical', 'atomized', 'atomism']
attempt ['attempt', 'attempting', 'attempts', 'attempted']
black ['black', 'blackness', 'blacking']
boson ['boson', 'bosons', 'bosonic', 'bosonization', 'bosonized', 'bosonize', 'bosonizing']
charg ['charge', 'charges', 'charged', 'charging', 'chargs', 'charg']
circuit ['circuit', 'circuits', 'circuital', 'circuited', 'circuitous', 'circuitation']
circular ['circular', 'circularly', 'circularity', 'circularization', 'circularize']
classic ['classical', 'classic', 'classics', 'classicalness']
conduct ['conducting', 'conduction', 'conductivity', 'conductive', 'conduct', 'conducted', 'conductance', 'conducts', 'conductivities', 'c

In [140]:
#word_index = tfidf_matrix[:,terms.index("allel")].nonzero()[0].tolist()
#list(combiny[i] for i in word_index[:5])

In [237]:
dst = 1-cosine_similarity(tfidf_matrix)
#type(dst)

In [215]:
#corr = pd.DataFrame(dst)
#corr.to_csv("corr.csv")

In [ ]:
#count1.iloc[0:20,:]


In [240]:
num_clusters = 100;
km = KMeans(n_clusters = num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()
joblib.dump(km,  'doc_cluster.pkl')


Wall time: 10.9 s


['doc_cluster.pkl', 'doc_cluster.pkl_01.npy', 'doc_cluster.pkl_02.npy']

In [241]:
order_centroids = km.cluster_centers_.argsort()[:,::-1]

In [147]:
#order_centroids.shape

(50, 91)

In [242]:
m = pd.Series(clusters)
biology["cluster"] = m.values

In [243]:
def top_words(x):
    ind = order_centroids[x,:6].tolist()
    list = []
    for inw in ind:
        list.append(lookup2.ix[terms[inw]]['word'])
    return list

def top_stemmed(x):
    ind = order_centroids[x,:6].tolist()
    return [terms[inw] for inw in ind]
    

In [244]:
for i in range(100):
    print(i)
    print(top_words(i))
    print(top_stemmed(i))

0
['species', 'human', 'biology', 'evolution', 'organism', 'environment']
['speci', 'human', 'biolog', 'evolut', 'organ', 'environ']
1
['immune', 'system', 'cells', 'body', 'virus', 'disease']
['immun', 'system', 'cell', 'bodi', 'virus', 'diseas']
2
['human', 'species', 'biology', 'evolve', 'sense', 'evolution']
['human', 'speci', 'biolog', 'evolv', 'sens', 'evolut']
3
['brain', 'biology', 'person', 'human', 'cells', 'physiological']
['brain', 'biolog', 'person', 'human', 'cell', 'physiolog']
4
['gene', 'sequence', 'genome', 'protein', 'dna', 'rna']
['gene', 'sequenc', 'genom', 'protein', 'dna', 'rna']
5
['sense', 'field', 'environment', 'adaptation', 'molecular', 'parent']
['sens', 'field', 'environ', 'adapt', 'molecular', 'parent']
6
['dna', 'cells', 'human', 'sequence', 'virus', 'genetic']
['dna', 'cell', 'human', 'sequenc', 'virus', 'genet']
7
['eye', 'human', 'brain', 'person', 'biology', 'body']
['eye', 'human', 'brain', 'person', 'biolog', 'bodi']
8
['blood', 'cells', 'body', 'h

In [245]:
biology["pred"] = biology["cluster"].map(top_stemmed)

In [246]:
biology.to_csv("pred1.csv")